# Prédiction du Churn Client - MLflow Organisé

## Du Chaos à la Reproductibilité

Dans le notebook précédent (`01_messy_notebook.ipynb`), nous avons vécu le chaos typique du data science :
- Plusieurs expériences sans tracking
- Métriques éparpillées dans les cellules
- "C'était lequel le meilleur modèle déjà ?"
- Pas de versioning ni de reproductibilité

**Dans ce notebook, nous résolvons ces problèmes avec MLflow :**
- Tracking d'expériences (paramètres, métriques, artefacts)
- Versioning de modèles et registry
- Comparaison facile entre les runs
- Résultats reproductibles

---

## 1. Setup et Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# NOUVEAU : imports MLflow
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

print(f"Version MLflow : {mlflow.__version__}")

## 2. Configuration MLflow

MLflow doit savoir :
1. **Où stocker les données de tracking** (tracking URI)
2. **Dans quelle expérience logger** (nom de l'expérience)

**Pour ce workshop :** Démarrez d'abord le serveur MLflow avec `cd docker && docker-compose up -d`

In [ ]:
# Configurer le tracking MLflow
# Se connecter au serveur MLflow hébergé dans Docker (démarrer avec : docker-compose up -d)
mlflow.set_tracking_uri("http://localhost:5000")

# Créer ou récupérer l'expérience
experiment_name = "customer-churn-prediction"
mlflow.set_experiment(experiment_name)

print(f"Tracking URI : {mlflow.get_tracking_uri()}")
print(f"Expérience : {experiment_name}")

# Tester la connexion avec un simple ping
try:
    import requests
    response = requests.get(f"{mlflow.get_tracking_uri()}/health")
    print(f"✅ Serveur MLflow répond : {response.status_code}")
except Exception as e:
    print(f"❌ Impossible de se connecter au serveur MLflow : {e}")

## 3. Charger les données

On charge les données clients depuis le CSV. Exécutez `python generate_sample_data.py` d'abord si le fichier n'existe pas.

In [ ]:
# Charger les données clients depuis le CSV
df = pd.read_csv('../data/customer_data.csv')

print(f"Dimensions du dataset : {df.shape}")
print(f"Taux de churn : {df['churned'].mean():.2%}")
df.head()

## 4. Feature Engineering

Mêmes features qu'avant - nous allons tracker quelles features nous utilisons dans MLflow.

In [ ]:
def engineer_features(df):
    """Créer des features engineerées - encapsulé dans une fonction pour la reproductibilité"""
    df = df.copy()
    
    # Features de ratio
    df['recency_frequency_ratio'] = df['recency_days'] / (df['frequency'] + 1)
    df['monetary_per_order'] = df['monetary_value'] / (df['total_orders'] + 1)
    df['order_frequency'] = df['total_orders'] / (df['days_since_signup'] + 1)
    df['support_per_order'] = df['support_tickets'] / (df['total_orders'] + 1)
    
    # Score RFM
    df['r_score'] = pd.qcut(df['recency_days'], q=5, labels=[5, 4, 3, 2, 1]).astype(int)
    df['f_score'] = pd.qcut(df['frequency'].rank(method='first'), q=5, labels=[1, 2, 3, 4, 5]).astype(int)
    df['m_score'] = pd.qcut(df['monetary_value'].rank(method='first'), q=5, labels=[1, 2, 3, 4, 5]).astype(int)
    df['rfm_score'] = df['r_score'] + df['f_score'] + df['m_score']
    
    return df

# Appliquer le feature engineering
df = engineer_features(df)
print(f"Features après engineering : {df.shape[1]} colonnes")

In [ ]:
# Définir les colonnes de features (on va logger ça dans MLflow)
FEATURE_COLS = [
    'recency_days', 'frequency', 'monetary_value', 'avg_order_value',
    'days_since_signup', 'total_orders', 'support_tickets', 'age',
    'recency_frequency_ratio', 'monetary_per_order', 'order_frequency',
    'support_per_order', 'rfm_score'
]

X = df[FEATURE_COLS]
y = df['churned']

# Séparation train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Échantillons d'entraînement : {len(X_train)}")
print(f"Échantillons de test : {len(X_test)}")

## 5. Tracking d'expériences MLflow

Maintenant la magie opère ! Au lieu d'afficher les métriques et de les oublier, nous **loggons tout dans MLflow**.

### Concepts clés MLflow :
- **Run** : Une exécution unique de votre code (une expérience)
- **Parameters** : Les entrées de votre modèle (hyperparamètres)
- **Metrics** : Les sorties/résultats (accuracy, F1, etc.)
- **Artifacts** : Les fichiers (modèles, graphiques, échantillons de données)
- **Tags** : Les métadonnées (auteur, version, notes)

### 5.1 Exemple de logging manuel

**Ce qu'on logge :**
- **Parameters** : Hyperparamètres, nombre de features, tailles train/test
- **Metrics** : Accuracy, precision, recall, F1
- **Artifacts** : Modèle, matrice de confusion, importance des features
- **Evaluation Tables** : Tableau détaillé des prédictions
- **Dataset** : Info sur les données d'entraînement (comme autolog fait)

**Note :** Assurez-vous que l'autologging est désactivé avant le logging manuel pour éviter les conflits.

In [ ]:
# S'assurer que l'autologging est désactivé pour éviter les conflits
mlflow.sklearn.autolog(disable=True)

import tempfile
import os

def train_and_log_model(model, model_name, X_train, X_test, y_train, y_test, params=None):
    """
    Entraîner un modèle et logger tout dans MLflow.
    Inclut : paramètres, métriques, modèle, artefacts, résultats d'évaluation et dataset.
    """
    with mlflow.start_run(run_name=model_name):
        # Logger les tags (métadonnées)
        mlflow.set_tag("model_type", model_name)
        mlflow.set_tag("author", "workshop")
        mlflow.set_tag("stage", "experimentation")
        
        # === Logger le dataset (comme autolog fait) ===
        train_data = X_train.copy()
        train_data['target'] = y_train.values
        dataset = mlflow.data.from_pandas(
            train_data, 
            source="../data/customer_data.csv",
            name="customer_churn_training",
            targets="target"
        )
        mlflow.log_input(dataset, context="training")
        
        # Logger les paramètres
        if params:
            mlflow.log_params(params)
        mlflow.log_param("n_features", len(FEATURE_COLS))
        mlflow.log_param("train_size", len(X_train))
        mlflow.log_param("test_size", len(X_test))
        
        # Entraîner le modèle
        model.fit(X_train, y_train)
        
        # Faire des prédictions
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
        
        # Calculer et logger les métriques
        metrics = {
            "accuracy": accuracy_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred),
            "recall": recall_score(y_test, y_pred),
            "f1": f1_score(y_test, y_pred)
        }
        mlflow.log_metrics(metrics)
        
        # Logger les résultats d'évaluation en CSV avec tempfile (pas de fichier local laissé)
        eval_df = pd.DataFrame({
            'actual': y_test.values,
            'predicted': y_pred,
            'probability': y_proba if y_proba is not None else [None] * len(y_pred),
            'correct': (y_test.values == y_pred).astype(int)
        })
        with tempfile.TemporaryDirectory() as tmpdir:
            csv_path = os.path.join(tmpdir, "evaluation_results.csv")
            eval_df.to_csv(csv_path, index=False)
            mlflow.log_artifact(csv_path)
        
        # Logger le modèle
        mlflow.sklearn.log_model(model, name="model")
        
        # Logger l'importance des features (si disponible) - avec log_figure (pas de fichier local)
        if hasattr(model, 'feature_importances_'):
            fig, ax = plt.subplots(figsize=(10, 6))
            feat_imp = pd.Series(model.feature_importances_, index=FEATURE_COLS)
            feat_imp.sort_values().plot(kind='barh', ax=ax)
            ax.set_title(f'Importance des Features - {model_name}')
            plt.tight_layout()
            mlflow.log_figure(fig, "feature_importance.png")
            plt.close()
        
        # Logger la matrice de confusion - avec log_figure (pas de fichier local)
        cm = confusion_matrix(y_test, y_pred)
        fig, ax = plt.subplots(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
        ax.set_xlabel('Prédit')
        ax.set_ylabel('Réel')
        ax.set_title(f'Matrice de Confusion - {model_name}')
        plt.tight_layout()
        mlflow.log_figure(fig, "confusion_matrix.png")
        plt.close()
        
        run_id = mlflow.active_run().info.run_id
        print(f"Modèle : {model_name} | Accuracy : {metrics['accuracy']:.4f} | F1 : {metrics['f1']:.4f}")
        print(f"Loggé : modèle, dataset, matrice de confusion, CSV d'évaluation")
        
        return run_id, metrics

### 5.2 Lancer les expériences

### Artefacts de prétraitement

**Important :** Les modèles nécessitant un prétraitement (scaling, encoding) doivent sauvegarder ces objets comme artefacts !

- Au moment de l'inférence, vous avez besoin du **même** scaler utilisé pendant l'entraînement
- Sans ça, les prédictions seront incorrectes (mauvaise échelle = mauvais résultats)

In [ ]:
# Expérience 1 : Régression Logistique (avec artefact de prétraitement)
import joblib

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

with mlflow.start_run(run_name="LogisticRegression") as run:
    # Logger les tags
    mlflow.set_tag("model_type", "LogisticRegression")
    mlflow.set_tag("author", "workshop")
    mlflow.set_tag("stage", "experimentation")
    mlflow.set_tag("requires_scaling", "true")
    
    # === Logger le dataset (comme autolog fait) ===
    train_data = X_train.copy()
    train_data['target'] = y_train.values
    dataset = mlflow.data.from_pandas(
        train_data,
        source="../data/customer_data.csv",
        name="customer_churn_training",
        targets="target"
    )
    mlflow.log_input(dataset, context="training")
    
    # Sauvegarder le scaler comme artefact avec tempfile (pas de fichier local laissé)
    with tempfile.TemporaryDirectory() as tmpdir:
        scaler_path = os.path.join(tmpdir, "scaler.pkl")
        joblib.dump(scaler, scaler_path)
        mlflow.log_artifact(scaler_path, artifact_path="preprocessing")
    
    # Logger les paramètres
    mlflow.log_params({
        "max_iter": 1000,
        "solver": "lbfgs",
        "n_features": len(FEATURE_COLS),
        "train_size": len(X_train_scaled),
        "test_size": len(X_test_scaled)
    })
    
    # Entraîner le modèle
    lr = LogisticRegression(random_state=42, max_iter=1000)
    lr.fit(X_train_scaled, y_train)
    
    # Faire des prédictions
    y_pred = lr.predict(X_test_scaled)
    y_proba = lr.predict_proba(X_test_scaled)[:, 1]
    
    # Calculer et logger les métriques
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred)
    }
    mlflow.log_metrics(metrics)
    
    # Logger les résultats d'évaluation en CSV avec tempfile (pas de fichier local laissé)
    eval_df = pd.DataFrame({
        'actual': y_test.values,
        'predicted': y_pred,
        'probability': y_proba,
        'correct': (y_test.values == y_pred).astype(int)
    })
    with tempfile.TemporaryDirectory() as tmpdir:
        csv_path = os.path.join(tmpdir, "evaluation_results.csv")
        eval_df.to_csv(csv_path, index=False)
        mlflow.log_artifact(csv_path)
    
    # Logger le modèle (utiliser name= au lieu de artifact_path= déprécié)
    mlflow.sklearn.log_model(lr, name="model")
    
    # Logger la matrice de confusion - avec log_figure (pas de fichier local)
    cm = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
    ax.set_xlabel('Prédit')
    ax.set_ylabel('Réel')
    ax.set_title('Matrice de Confusion - LogisticRegression')
    plt.tight_layout()
    mlflow.log_figure(fig, "confusion_matrix.png")
    plt.close()
    
    lr_run_id = run.info.run_id
    lr_metrics = metrics
    
    print(f"Modèle : LogisticRegression | Accuracy : {metrics['accuracy']:.4f} | F1 : {metrics['f1']:.4f}")
    print(f"Loggé : modèle, dataset, scaler, matrice de confusion, CSV d'évaluation")

In [ ]:
# Expérience 2 : Random Forest (paramètres par défaut)
rf1 = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf1_run_id, rf1_metrics = train_and_log_model(
    rf1,
    "RandomForest_v1",
    X_train, X_test, y_train, y_test,
    params={"n_estimators": 100, "max_depth": None, "min_samples_split": 2}
)

In [ ]:
# Expérience 3 : Random Forest (paramètres optimisés)
rf2 = RandomForestClassifier(
    n_estimators=200, 
    max_depth=10, 
    min_samples_split=20,
    random_state=42, 
    n_jobs=-1
)
rf2_run_id, rf2_metrics = train_and_log_model(
    rf2,
    "RandomForest_v2_tuned",
    X_train, X_test, y_train, y_test,
    params={"n_estimators": 200, "max_depth": 10, "min_samples_split": 20}
)

In [ ]:
# Expérience 4 : Gradient Boosting
gb = GradientBoostingClassifier(
    n_estimators=100, 
    learning_rate=0.1, 
    max_depth=5, 
    random_state=42
)
gb_run_id, gb_metrics = train_and_log_model(
    gb,
    "GradientBoosting",
    X_train, X_test, y_train, y_test,
    params={"n_estimators": 100, "learning_rate": 0.1, "max_depth": 5}
)

In [ ]:
# Expérience 5 : Gradient Boosting (paramètres différents)
gb2 = GradientBoostingClassifier(
    n_estimators=200, 
    learning_rate=0.05, 
    max_depth=4, 
    random_state=42
)
gb2_run_id, gb2_metrics = train_and_log_model(
    gb2,
    "GradientBoosting_v2",
    X_train, X_test, y_train, y_test,
    params={"n_estimators": 200, "learning_rate": 0.05, "max_depth": 4}
)

### 5.3 Alternative : Autologging

MLflow peut automatiquement logger les paramètres et métriques pour les frameworks supportés !

**L'autolog capture :**
- **Tous les paramètres** (y compris les défauts comme `bootstrap=True`, `ccp_alpha=0.0`, etc.)
- **Métriques d'entraînement** avec des noms différents : `training_accuracy_score`, `training_f1_score`, etc.
- **Artefact du modèle** et signature
- **Info du dataset** (schéma d'entrée, lignes d'exemple)
- **Importance des features** (pour les modèles basés sur les arbres)

**Compromis :**
| Logging Manuel | Autologging |
|----------------|-------------|
| Contrôle total sur ce qui est loggé | Logge tout automatiquement |
| Noms de métriques cohérents (`accuracy`) | Noms spécifiques au framework (`training_accuracy_score`) |
| Liste de paramètres propre | Tous les paramètres (y compris les défauts) |
| Plus de code à écrire | Zéro code de logging nécessaire |

In [ ]:
# Activer l'autologging pour sklearn
mlflow.sklearn.autolog()

# Maintenant entraîner sans logging manuel - MLflow capture tout !
with mlflow.start_run(run_name="RandomForest_autolog"):
    mlflow.set_tag("logging_method", "autolog")
    
    rf_auto = RandomForestClassifier(
        n_estimators=150, 
        max_depth=12, 
        random_state=42,
        n_jobs=-1
    )
    rf_auto.fit(X_train, y_train)
    
    # L'autolog capture automatiquement :
    # - Tous les paramètres du modèle (y compris les défauts !)
    # - Métriques d'entraînement (training_accuracy_score, etc.) - SUR LES DONNÉES D'ENTRAÎNEMENT
    # - Artefact du modèle
    # - Importance des features
    
    # IMPORTANT : Les métriques autolog sont sur les données d'ENTRAÎNEMENT !
    # Pour une comparaison juste, on logge aussi les métriques de TEST manuellement :
    y_pred = rf_auto.predict(X_test)
    mlflow.log_metrics({
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred)
    })
    
    print("L'autologging a capturé tous les paramètres et le modèle automatiquement !")
    print(f"Note : les métriques training_* sont sur les données d'entraînement (seront plus élevées)")
    print(f"      accuracy/f1/etc sont sur les données de test (comparables aux autres runs)")

# Désactiver l'autolog pour le contrôle manuel
mlflow.sklearn.autolog(disable=True)

## 6. Comparer les expériences

Maintenant voyons la puissance de MLflow - comparer toutes nos expériences !

In [ ]:
# Récupérer tous les runs de notre expérience
client = MlflowClient()
experiment = client.get_experiment_by_name(experiment_name)

runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["metrics.f1 DESC"]
)

print(f"Total des runs : {len(runs)}\n")
print("Toutes les expériences classées par score F1 (métriques de TEST) :")
print("=" * 80)

results = []
for run in runs:
    if run.data.metrics:
        metrics = run.data.metrics
        
        # Prioriser les métriques de TEST (accuracy, f1, etc.)
        # Fallback sur les métriques d'entraînement seulement si les métriques de test n'existent pas
        accuracy = metrics.get('accuracy', 0)
        precision = metrics.get('precision', 0)
        recall = metrics.get('recall', 0)
        f1 = metrics.get('f1', 0)
        
        # Ignorer les runs qui n'ont pas de métriques de test
        if f1 == 0 and 'training_f1_score' in metrics:
            # Ce run n'a que des métriques d'entraînement - afficher mais signaler
            accuracy = metrics.get('training_accuracy_score', 0)
            f1 = metrics.get('training_f1_score', 0)
            precision = metrics.get('training_precision_score', 0)
            recall = metrics.get('training_recall_score', 0)
            metric_type = "TRAIN (non comparable !)"
        else:
            metric_type = "test"
        
        logging_method = run.data.tags.get('logging_method', 'manual')
        
        results.append({
            'run_name': run.info.run_name,
            'run_id': run.info.run_id[:8],
            'metric_type': metric_type,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1
        })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('f1', ascending=False).reset_index(drop=True)

print("\nNote : Tous les runs utilisent maintenant les métriques de TEST pour une comparaison juste.")
print("      Les runs avec uniquement des métriques d'entraînement sont signalés.\n")
results_df

In [ ]:
# Visualiser la comparaison
if len(results_df) > 0:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    x = range(len(results_df))
    width = 0.2
    
    ax.bar([i - 1.5*width for i in x], results_df['accuracy'], width, label='Accuracy')
    ax.bar([i - 0.5*width for i in x], results_df['precision'], width, label='Precision')
    ax.bar([i + 0.5*width for i in x], results_df['recall'], width, label='Recall')
    ax.bar([i + 1.5*width for i in x], results_df['f1'], width, label='F1')
    
    ax.set_xlabel('Modèle')
    ax.set_ylabel('Score')
    ax.set_title('Comparaison des Modèles (depuis MLflow)')
    ax.set_xticks(x)
    ax.set_xticklabels(results_df['run_name'], rotation=45, ha='right')
    ax.legend()
    ax.set_ylim(0, 1)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Trouver le meilleur modèle
if len(results_df) > 0:
    best_run = results_df.loc[results_df['f1'].idxmax()]
    print(f"Meilleur modèle par score F1 : {best_run['run_name']}")
    print(f"Score F1 : {best_run['f1']:.4f}")

## 7. Model Registry

Maintenant que nous avons trouvé notre meilleur modèle, **enregistrons-le**. Le Model Registry fournit :
- Contrôle de version pour les modèles
- Transitions d'état (Staging → Production)
- Lignage du modèle (quel run a créé ce modèle)

In [ ]:
# Obtenir l'ID du meilleur run (par score F1)
best_runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    filter_string="metrics.f1 > 0",
    order_by=["metrics.f1 DESC"],
    max_results=1
)

if best_runs:
    best_run = best_runs[0]
    best_run_id = best_run.info.run_id
    print(f"Meilleur run : {best_run.info.run_name}")
    print(f"Run ID : {best_run_id}")
    print(f"F1 : {best_run.data.metrics.get('f1', 0):.4f}")

In [ ]:
# Enregistrer le meilleur modèle
model_name = "customer-churn-classifier"

if best_runs:
    model_uri = f"runs:/{best_run_id}/model"
    
    # Enregistrer le modèle
    registered_model = mlflow.register_model(
        model_uri=model_uri,
        name=model_name
    )
    
    print(f"Modèle enregistré : {model_name}")
    print(f"Version : {registered_model.version}")

In [ ]:
# Voir les versions du modèle enregistré
try:
    versions = client.search_model_versions(f"name='{model_name}'")
    print(f"\nVersions enregistrées de '{model_name}' :")
    print("=" * 60)
    for v in versions:
        print(f"Version {v.version} : Run ID {v.run_id[:8]}... | Statut : {v.status}")
except Exception as e:
    print(f"Aucun modèle enregistré trouvé : {e}")

## 8. Charger et utiliser le modèle enregistré

En production, vous chargez les modèles depuis le registry - pas depuis des fichiers locaux !

In [ ]:
# Charger le modèle depuis le registry avec les artefacts de prétraitement
import joblib
import tempfile

try:
    # Charger la dernière version
    model_uri = f"models:/{model_name}/latest"
    loaded_model = mlflow.sklearn.load_model(model_uri)
    
    print(f"Modèle chargé : {model_name}")
    print(f"Type de modèle : {type(loaded_model).__name__}")
    
    # Vérifier si ce modèle nécessite un prétraitement
    client = MlflowClient()
    model_version = client.get_latest_versions(model_name, stages=[])[0]
    run_id = model_version.run_id
    run = client.get_run(run_id)
    
    requires_scaling = run.data.tags.get("requires_scaling", "false") == "true"
    
    if requires_scaling:
        print(f"\n⚠️  Ce modèle nécessite un prétraitement (scaling)")
        
        # Télécharger l'artefact du scaler
        artifact_path = client.download_artifacts(run_id, "preprocessing/scaler.pkl")
        scaler = joblib.load(artifact_path)
        print(f"✅ Scaler chargé depuis les artefacts")
        
        # Appliquer le prétraitement avant la prédiction
        X_test_scaled = scaler.transform(X_test[:5])
        sample_predictions = loaded_model.predict(X_test_scaled)
        sample_probas = loaded_model.predict_proba(X_test_scaled)[:, 1]
        
        print("\nExemples de prédictions (avec prétraitement correct) :")
        for i, (pred, proba) in enumerate(zip(sample_predictions, sample_probas)):
            actual = y_test.iloc[i]
            print(f"  Client {i+1} : Prédit={pred}, Réel={actual}, Probabilité={proba:.2%}")
    else:
        # Les modèles basés sur les arbres n'ont pas besoin de scaling
        print(f"\n✅ Ce modèle n'a pas besoin de prétraitement")
        sample_predictions = loaded_model.predict(X_test[:5])
        sample_probas = loaded_model.predict_proba(X_test[:5])[:, 1]
        
        print("\nExemples de prédictions :")
        for i, (pred, proba) in enumerate(zip(sample_predictions, sample_probas)):
            actual = y_test.iloc[i]
            print(f"  Client {i+1} : Prédit={pred}, Réel={actual}, Probabilité={proba:.2%}")
        
except Exception as e:
    print(f"Impossible de charger depuis le registry : {e}")
    print("C'est attendu si c'est la première exécution.")

### Point clé à retenir

Remarquez comment le code ci-dessus :
1. **Vérifie les tags du modèle** pour voir si un prétraitement est nécessaire
2. **Télécharge l'artefact du scaler** depuis MLflow
3. **Applique la même transformation** utilisée pendant l'entraînement

C'est le **pattern prêt pour la production** :
```python
# Pendant l'entraînement
scaler.fit_transform(X_train)  # Apprendre des données d'entraînement
mlflow.log_artifact("scaler.pkl")  # Le sauvegarder !

# Pendant l'inférence  
scaler = load_artifact("scaler.pkl")  # Le charger !
scaler.transform(X_new)  # Appliquer (ne jamais refaire fit !)
```

**Erreur courante :** Refaire fit du scaler sur de nouvelles données → échelle différente → prédictions incorrectes !

## 9. Voir l'UI MLflow

Le serveur MLflow tourne dans Docker. Ouvrez http://localhost:5000 dans votre navigateur.

Vous verrez :
- Toutes les expériences et runs
- Comparaison de paramètres et métriques
- Navigateur d'artefacts (modèles, graphiques)
- Model registry

**Note :** Si vous n'avez pas encore démarré le serveur, exécutez :
```bash
cd docker && docker-compose up -d
```

---

## Résumé : Ce que MLflow a résolu

| Problème (Notebook chaotique) | Solution (MLflow) |
|-------------------------------|-------------------|
| "C'était lequel le meilleur modèle ?" | Requêter les runs par métriques |
| Métriques éparpillées dans les cellules | Stockage centralisé des métriques |
| Pas de versioning | Model Registry avec versions |
| Impossible de reproduire | Run ID lié au code/params exact |
| Comparaison manuelle | UI de comparaison automatique |
| Fichiers de modèles perdus | Artefacts stockés avec le run |

## Ce qui reste manuel ?

MLflow ne résout pas tout :
- **Planification** : Comment exécuter ça quotidiennement ?
- **Dépendances** : Et si le chargement des données échoue ?
- **Alertes** : Comment savoir si l'entraînement a échoué ?
- **CI/CD** : Comment automatiser le déploiement ?

**Prochaine étape** : Orchestrateurs (Prefect, Airflow, Dagster) pour automatiser le pipeline !

---

## Exercices

1. **Ajouter un nouveau modèle** : Essayez XGBoost ou LightGBM et loggez-le dans MLflow
2. **Recherche d'hyperparamètres** : Lancez plusieurs expériences avec différents paramètres
3. **Métriques personnalisées** : Loggez une métrique personnalisée (ex : AUC-ROC)
4. **Comparaison de modèles** : Utilisez l'UI MLflow pour comparer vos expériences
5. **Workflow du registry** : Enregistrez votre meilleur modèle et rechargez-le